# Fluorescent Image Analysis Test Script 
20/03/25 Written By Fraser Shields

## 1. Import Images and Combine Channels
- Used for images with seperate channels

In [25]:
import os
import glob
import numpy as np
import skimage.io as io
import skimage.exposure

import cv2



## Combining Channels Into Composites

In [12]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Path to the image directory
image_dir = r"C:\Users\mbgm4fs3\OneDrive - The University of Manchester\PhD\Experimental\Data\5. Mechanical Stimulation\Primary\Imaging\HA\all_images"

# Create a new folder for combined images
combined_dir = os.path.join(image_dir, 'combined')
os.makedirs(combined_dir, exist_ok=True)

# Create a list of all filenames in the directory
filenames = os.listdir(image_dir)

# Separate the filenames based on the suffixes C1 and C2
c1_images = [f for f in filenames if f.endswith('C1.tif')]
c2_images = [f for f in filenames if f.endswith('C2.tif')]

# Sort the filenames to ensure they're in the correct order (assuming they match)
c1_images.sort()
c2_images.sort()

# Check that the lists are of the same length
if len(c1_images) != len(c2_images):
    raise ValueError("The number of C1 and C2 images do not match!")

# Linear mapping function (similar to ImageJ's brightness/contrast)
def apply_brightness_contrast(image, min_val, max_val):
    # Create a copy to avoid modifying the original
    adjusted = image.copy().astype(float)
    
    # Apply linear mapping
    adjusted = (adjusted - min_val) / (max_val - min_val)
    
    # Clip values to 0-1 range
    adjusted[adjusted < 0] = 0
    adjusted[adjusted > 1] = 1
    
    return adjusted

# Loop through each pair of images (C1 and C2)
for c1_img, c2_img in zip(c1_images, c2_images):
    # Load the images
    c1_path = os.path.join(image_dir, c1_img)
    c2_path = os.path.join(image_dir, c2_img)
    
    c1 = np.array(Image.open(c1_path))
    c2 = np.array(Image.open(c2_path))
    
    # # Apply brightness/contrast adjustment (more like ImageJ)
    c1_adjusted = apply_brightness_contrast(c1, 150, 600)  # Apply to channel C1 (cyan)
    c2_adjusted = apply_brightness_contrast(c2, 150, 5000)  # Apply to channel C2 (red)

    
    # Create RGB image (initialize with zeros)
    combined_image = np.zeros((c1.shape[0], c1.shape[1], 3))
    
    # Assign CYAN (0,1,1) for C1 channel
    combined_image[:, :, 0] = 0              # No Red for cyan
    combined_image[:, :, 1] = c1_adjusted    # Green component for cyan
    combined_image[:, :, 2] = c1_adjusted    # Blue component for cyan
    
    # Overlay RED (1,0,0) for C2 channel
    combined_image[:, :, 0] = c2_adjusted    # Red component
    
    # Convert to uint8 (0-255)
    combined_image_uint8 = (combined_image * 255).astype(np.uint8)
    
    # Convert to a PIL image and save it in the 'combined' folder
    combined_image_pil = Image.fromarray(combined_image_uint8)
    
    # Name without the C1 suffix
    base_name = os.path.splitext(c1_img)[0]
    if base_name.endswith('C1'):
        base_name = base_name[:-2]
        
    final_name = f"combined_{base_name}.tif"
    combined_image_pil.save(os.path.join(combined_dir, final_name))
    print(f"Saved combined image: {final_name}")

Saved combined image: combined_Sample_mscw084_negative_n1W0002F0003T0001Z000.tif
Saved combined image: combined_Sample_mscw084_negative_n2W0002F0002T0001Z000.tif
Saved combined image: combined_Sample_mscw084_negative_n3W0002F0001T0001Z000.tif
Saved combined image: combined_Sample_mscw213_2224_positiveW0001F0001T0001Z000.tif
Saved combined image: combined_Sample_mscw213_2224_positive_n2W0001F0002T0001Z000.tif
Saved combined image: combined_Sample_mscw213_2224_positive_n3W0001F0003T0001Z000.tif
Saved combined image: combined_Sample_w137_d14_400_con_t6_n1W0004F0001T0001Z000.tif
Saved combined image: combined_Sample_w137_d14_400_con_t6_n2W0004F0002T0001Z000.tif
Saved combined image: combined_Sample_w137_d14_400_con_t6_n3W0004F0003T0001Z000.tif
Saved combined image: combined_Sample_w137_d14_800_con_t6_n1W0001F0001T0001Z000.tif
Saved combined image: combined_Sample_w137_d14_800_con_t6_n2W0001F0002T0001Z000.tif
Saved combined image: combined_Sample_w137_d14_800_con_t6_n3W0001F0003T0001Z000.ti

### Without thresholding or linear mapping

In [30]:
import os
import numpy as np
import tifffile as tiff
from PIL import Image

# Path to the image directory
image_dir = r"C:\Users\mbgm4fs3\OneDrive - The University of Manchester\PhD\Experimental\Data\5. Mechanical Stimulation\Primary\Imaging\ACAN\all_images"

# Create a new folder for combined images
combined_dir = os.path.join(image_dir, 'combined')
os.makedirs(combined_dir, exist_ok=True)

# Define the fixed global intensity range
min_intensity = 0       # Typically 0
max_intensity = 5000    # Adjust based on expected signal intensity

# Create a list of all filenames in the directory
filenames = os.listdir(image_dir)

# Separate the filenames based on the suffixes C1 and C2
c1_images = [f for f in filenames if f.endswith('C1.tif')]
c2_images = [f for f in filenames if f.endswith('C2.tif')]

# Sort the filenames to ensure they're in the correct order (assuming they match)
c1_images.sort()
c2_images.sort()

# Check that the lists are of the same length
if len(c1_images) != len(c2_images):
    raise ValueError("The number of C1 and C2 images do not match!")


# Loop through each pair of images (C1 and C2)
for c1_img, c2_img in zip(c1_images, c2_images):
    # Load the images
    c1_path = os.path.join(image_dir, c1_img)
    c2_path = os.path.join(image_dir, c2_img)

    c1 = np.array(Image.open(c1_path))
    c2 = np.array(Image.open(c2_path))

    # Ensure images are 16-bit
    if c1.dtype != np.uint16 or c2.dtype != np.uint16:
        raise ValueError("Input images must be 16-bit TIFFs.")

    # Create an empty 16-bit 3-channel image
    combined_image = np.zeros((c1.shape[0], c1.shape[1], 3), dtype=np.uint16)

        # Assign CYAN (0,1,1) for C1 channel
    combined_image[:, :, 1] = c1  # Green component
    combined_image[:, :, 2] = c1  # Blue component

    # Assign RED (1,0,0) for C2 channel
    combined_image[:, :, 0] = c2  # Red component

    # Save using tifffile (True 16-bit TIFF)
    base_name = os.path.splitext(c1_img)[0]
    if base_name.endswith('C1'):
        base_name = base_name[:-2]

    final_name = f"combined_{base_name}.tif"
    tiff.imwrite(os.path.join(combined_dir, final_name), combined_image, dtype=np.uint16)

    print(f"Saved combined image: {final_name}")


Saved combined image: combined_Sample_W0001F0002T0001Z000.tif
Saved combined image: combined_msc_w178_IGg_n1W0003F0001T0001Z000.tif
Saved combined image: combined_msc_w178_IGg_n2W0003F0002T0001Z000.tif
Saved combined image: combined_msc_w178_IGg_n3W0003F0003T0001Z000.tif
Saved combined image: combined_msc_w178_negative_n1W0004F0001T0001Z000.tif
Saved combined image: combined_msc_w178_negative_n2W0004F0002T0001Z000.tif
Saved combined image: combined_msc_w178_negative_n3W0004F0003T0001Z000.tif
Saved combined image: combined_msc_w178_positive_n1W0001F0003T0001Z000.tif
Saved combined image: combined_msc_w178_positive_n2W0001F0002T0001Z000.tif
Saved combined image: combined_msc_w178_positive_n3W0001F0001T0001Z000.tif
Saved combined image: combined_w137_d14_400_con_n1W0003F0001T0001Z000.tif
Saved combined image: combined_w137_d14_400_con_n2W0003F0002T0001Z000.tif
Saved combined image: combined_w137_d14_400_con_n3W0003F0003T0001Z000.tif
Saved combined image: combined_w137_d14_800_con_n1W0002F

## Splitting Composites into channels 

In [ ]:
import os
import numpy as np
from PIL import Image
import tifffile

# Path to the image directory
base_dir = r"C:\Users\mbgm4fs3\OneDrive - The University of Manchester\PhD\Experimental\Data\5. Mechanical Stimulation\Primary\Imaging\HA\all_images"
image_dir = os.path.join(base_dir, "w184")

# Create output folders
output_dir = os.path.join(image_dir, 'processed')
os.makedirs(output_dir, exist_ok=True)

split_dir = os.path.join(output_dir, 'split')
os.makedirs(split_dir, exist_ok=True)

combined_dir = os.path.join(output_dir, 'combined')
os.makedirs(combined_dir, exist_ok=True)

# Get all tif files in the directory
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith('.tif')]

# Linear mapping function (similar to ImageJ's brightness/contrast)
def apply_brightness_contrast(image, min_val, max_val):
    # Create a copy to avoid modifying the original
    adjusted = image.copy().astype(float)
    
    # Apply linear mapping
    adjusted = (adjusted - min_val) / (max_val - min_val)
    
    # Clip values to 0-1 range
    adjusted[adjusted < 0] = 0
    adjusted[adjusted > 1] = 1
    
    return adjusted

# Process each image
for img_file in image_files:
    # Skip if it's in a subdirectory we created
    if img_file in ['processed', 'split', 'combined']:
        continue
        
    print(f"Processing {img_file}...")
    
    # Load the image using tifffile which handles multi-channel TIFFs better
    img_path = os.path.join(image_dir, img_file)
    try:
        # Open the multi-channel TIFF
        with tifffile.TiffFile(img_path) as tif:
            # Check if this is a multi-page/multi-channel TIFF
            if len(tif.pages) >= 2:
                # Read the two channels separately
                channel2 = tif.pages[0].asarray()  # First channel (will be colored cyan)
                channel1 = tif.pages[1].asarray()  # Second channel (will be colored red)
                
                print(f"  Found multi-channel TIFF with shape: {channel1.shape}, {channel2.shape}")
            else:
                print(f"  Skipping {img_file} - not a multi-channel TIFF")
                continue
    except Exception as e:
        print(f"  Error opening {img_file}: {e}")
        continue
    
    # Save the split channels
    base_name = os.path.splitext(img_file)[0]
    
    # Save individual channels
    tifffile.imwrite(os.path.join(split_dir, f"{base_name}_C1.tif"), channel1)
    tifffile.imwrite(os.path.join(split_dir, f"{base_name}_C2.tif"), channel2)
    
    # Apply thresholds
    channel1_adjusted = apply_brightness_contrast(channel1, 150, 500)
    channel2_adjusted = apply_brightness_contrast(channel2, 150, 500)
    
    # Create a new RGB image for the colored channels
    height, width = channel1.shape
    combined = np.zeros((height, width, 3))
    
    # Assign CYAN (0,1,1) for channel1
    combined[:, :, 0] = 0                    # No Red for cyan
    combined[:, :, 1] = channel1_adjusted    # Green component for cyan
    combined[:, :, 2] = channel1_adjusted    # Blue component for cyan
    
    # Overlay RED (1,0,0) for channel2
    combined[:, :, 0] = channel2_adjusted    # Red component
    
    # Convert to uint8 for saving
    combined_uint8 = (combined * 255).astype(np.uint8)
    
    # Save the recombined image as a standard RGB TIFF
    combined_img = Image.fromarray(combined_uint8)
    combined_img.save(os.path.join(combined_dir, f"{base_name}_combined.tif"))
    
    print(f"  Completed processing {img_file}")

print("All images processed!")

Processing w128_d28_400_con_neg_n1.tif...
  Found multi-channel TIFF with shape: (2160, 2560), (2160, 2560)
  Completed processing w128_d28_400_con_neg_n1.tif
Processing w128_d28_400_con_neg_n2.tif...
  Found multi-channel TIFF with shape: (2160, 2560), (2160, 2560)
  Completed processing w128_d28_400_con_neg_n2.tif
Processing w128_d28_400_con_neg_n3.tif...
  Found multi-channel TIFF with shape: (2160, 2560), (2160, 2560)
  Completed processing w128_d28_400_con_neg_n3.tif
Processing w128_d28_800_con_neg_n1.tif...
  Found multi-channel TIFF with shape: (2160, 2560), (2160, 2560)
  Completed processing w128_d28_800_con_neg_n1.tif
Processing w128_d28_800_con_neg_n2.tif...
  Found multi-channel TIFF with shape: (2160, 2560), (2160, 2560)
  Completed processing w128_d28_800_con_neg_n2.tif
Processing w128_d28_800_con_neg_n3.tif...
  Found multi-channel TIFF with shape: (2160, 2560), (2160, 2560)
  Completed processing w128_d28_800_con_neg_n3.tif
Processing w184_d14_400_con_n1.tif...
  Found 